<br><font face="Times New Roman" size=5><div dir=ltr align=center>
<font color=blue size=8>
    Introduction to Machine Learning <br>
<font color=red size=5>
    Sharif University of Technology - Computer Engineering Department <br>
    Fall 2022<br> <br>
<font color=black size=6>
    Homework 2: Practical - Linear Regression
    </div>
<br><br>
<font size=4>
   **Name**: **MohammadAli MohammadKhani**<br>
   **Student ID**: **98102251**<br> <br>

<font face="Times New Roman" size=4><div dir=ltr>
# Problem 1: Linear Regression Model (40 + 30 optional points)
According to <a href="https://github.com/asharifiz/Introduction_to_Machine_Learning/blob/main/Jupyter_Notebooks/Chapter_02_Classical_Models/Linear%20regression.ipynb"><font face="Roboto">Linear Regression Notebook</font></a>, train a linear regression model on an arbitrary dataset. Explain your chosen dataset and split your data into train and test sets, then predict values for the test set using your trained model. Try to find the best hyperparameters for your model. (Using Lasso Regression, Ridge Regression or Elastic Net and comparing them will have extra optional points)
<br> Explain each step of your workflow.

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd
diabetes_dataset = datasets.load_diabetes(return_X_y=True, as_frame=True)

X, Y = diabetes_dataset[0], diabetes_dataset[1]      ## Loading Features (X) and targets (Y)

train_X = X.iloc[:200, :].to_numpy()


train_Y = Y.iloc[:200].to_numpy()
test_X = X.iloc[420: 442, :].to_numpy()
test_Y = Y.iloc[420: 442].to_numpy()      ### Splitting dataset into train and test in a proportion of about 80%

train_X = np.append(np.ones((200, 1)), train_X , axis=1)   ### Adding 1 to each row to include multiplying b 
test_X = np.append(np.ones((22, 1)), test_X , axis=1)   ### Adding 1 to each row to include multiplying b 


### DataSet Explanantion
This dataset has 10 featured, named below:
##### Age
##### Sex
##### BMI
##### BP
##### S1
##### S2
##### S3
##### S4
##### S5
##### S6

All of these features have numerical values, so we won't worry about nan values, also, they are related to physical and health-related issues, so we don't need to understand them all. This dataset consists of 2 parts, included in a tuple. The first part includes features and the second part is targets, which are consistent values, so we can pergorm regression on them. We train and test our model in 3 ways:
##### Dot Product and updating weights' vector using gradient descent: Doing this iterationally, we constantly make predictions using dot product and then we update our weights vector.
##### Closed Form: Using a closed form introduced in the next parts, we can simply obtain a finalized weights vector.
##### Using sklearn: In this scenario, we just import the needed functions from sklearn and call the fit() and predict() functions and get the loss.


In [2]:
W = np.random.rand(11, 1) ## Initializing random weights, one element more because of including bias.

def closed_solution(X, Y):
    W = np.dot((np.linalg.inv(np.dot(X.T,X))), np.dot(X.T,Y))  ## Find the weights using the formula  W = (X ^ (T) * X) ^ (-1) * X^(T)*Y
    return W

def make_predictions(X, W):
    prediction = np.dot(X, W)    #### Dot product to make predictions
    return prediction


def calculate_loss(pred, targ):
    residual = pred - targ
    return np.sum(residual * residual) / residual.size  ### Calculate MSE loss
    
def update_weights(old_weights, learning_rate, gradient):
    new_weights = old_weights - learning_rate * gradient
    return new_weights

In [3]:
def train_model_lr_gd(X, Y, W, n_epoches, lr):
    losses = []
    for i in range(n_epoches):
        predictions = make_predictions(X, W) ## Making Predictions
        error = predictions - Y # calculating the loss
        loss = calculate_loss(predictions, Y) # adding the loss to our loss list 
        losses.append(loss) # calculating gradients
        gradient = np.sum(error) / Y.size # updating weights and biases
        W = update_weights(W, lr, gradient)
        
    return W, losses

In [4]:
learning_rate = 0.009
n_epoches = 200
W, first_method_losses = train_model_lr_gd(train_X, train_Y, W, n_epoches, learning_rate)
print("The starting loss is: " + str(first_method_losses[0]))
print("The Final loss is: " + str(first_method_losses[len(first_method_losses) - 1]))

The starting loss is: 26853.760508514988
The Final loss is: 7058.199118602678


In [5]:
### Now We want to test the model

predictions = make_predictions(X=test_X, W=W)
print(calculate_loss(predictions, test_Y))


6785.660254752159


In [6]:
### Now lets make predictions using the closed form this time:

weights = closed_solution(train_X, train_Y)
predictions = make_predictions(train_X, weights)
print("Loss for train is: " + str(calculate_loss(predictions, train_Y)))

weights = closed_solution(test_X, test_Y)
predictions = make_predictions(test_X, weights)
print("Loss for test is: " + str(calculate_loss(predictions, test_Y)))

Loss for train is: 2698.1591490916035
Loss for test is: 1276.4517603050972


In [7]:
## Now lets do it by using sklearn perse:

from sklearn.linear_model import LinearRegression

model = LinearRegression()

model.fit(train_X, train_Y)
score = model.score(train_X, train_Y)
print(score)

0.5133662718598058


In [8]:
predictions = model.predict(test_X)
sklearn_loss = calculate_loss(predictions, test_Y)
print(sklearn_loss)

1764.9294214745664


## Comparison
comparing these 3 methods, as we can see, the loss for the third method (using sklearn) is the least. Then there is the closed form loss, and above all, the dot product method has the highest loss. So, the sklearn method is the best.

### Now We want to use Lasso and ElasticNet and Ridge, to observe what changes thet bring to out regression

In [9]:
from sklearn.linear_model import Lasso, ElasticNet, Ridge

lasso_regressior = Lasso(alpha=0.5)
lasso_regressior.fit(train_X, train_Y)
predictions = lasso_regressior.predict(test_X)
Lasso_loss = calculate_loss(predictions, test_Y)
print("Lasso loss is: " + str(Lasso_loss))


ElasticNet_regressior = ElasticNet(alpha=0.5)
ElasticNet_regressior.fit(train_X, train_Y)
predictions = ElasticNet_regressior.predict(test_X)
ElasticNet_loss = calculate_loss(predictions, test_Y)
print("ElasticNet loss is: " + str(ElasticNet_loss))



ridge_regressior = Ridge(alpha=0.5)
ridge_regressior.fit(train_X, train_Y)
predictions = ridge_regressior.predict(test_X)
ridge_loss = calculate_loss(predictions, test_Y)
print("Ridge loss is: " + str(ridge_loss))


Lasso loss is: 2192.4702930938747
ElasticNet loss is: 4868.80839043385
Ridge loss is: 2432.360867217826


# Conclusion
As we can see, The loss for Ridge is the lowest. Then Lasso and after that, ElasticNet has a higher loss. 